In [197]:
import openreview
# client = openreview.api.OpenReviewClient(baseurl='https://api2.openreview.net')
client = openreview.Client(baseurl='https://api.openreview.net')

conference_id = 'NeurIPS.cc/2022/Conference'
papers = client.get_all_notes(
    invitation=f'{conference_id}/-/Blind_Submission',
    details='directReplies', 
)


Getting V1 Notes: 100%|█████████▉| 2821/2824 [00:05<00:00, 509.13it/s]


In [198]:
papers[0].__dict__

{'id': 'zzDrPqn57DL',
 'original': 'HXF7e04ViJI',
 'number': 2310,
 'cdate': 1652737363731,
 'pdate': 1667239200000,
 'odate': None,
 'mdate': None,
 'tcdate': 1652737363731,
 'tmdate': 1720019111771,
 'ddate': None,
 'content': {'title': 'BEVFusion: A Simple and Robust LiDAR-Camera Fusion Framework',
  'authorids': ['~Tingting_Liang2',
   '~Hongwei_Xie1',
   '~Kaicheng_Yu1',
   '~Zhongyu_Xia1',
   '~Zhiwei_Lin1',
   '~Yongtao_Wang1',
   '~Tao_Tang4',
   '~Bing_Wang14',
   '~Zhi_Tang2'],
  'authors': ['Tingting Liang',
   'Hongwei Xie',
   'Kaicheng Yu',
   'Zhongyu Xia',
   'Zhiwei Lin',
   'Yongtao Wang',
   'Tao Tang',
   'Bing Wang',
   'Zhi Tang'],
  'keywords': ['3D Object Detection',
   "bird's eye view perception",
   'camera-lidar fusion'],
  'TL;DR': 'We introduce a simple LiDAR-camera fusion framework that overcomes the reliance of previous fusion methods on LiDAR through BEV space fusion.',
  'abstract': 'Fusing the camera and LiDAR information has become a de-facto standar

In [ ]:
# for NeurIPS 2022, the decision is stored in the /Decision invitation
import pandas as pd
import json
from datetime import datetime


# Function to convert timestamp to readable date
def convert_timestamp(ts):
    if ts is None:
        return None
    try:
        return datetime.fromtimestamp(ts / 1000).strftime('%Y-%m-%d %H:%M:%S')
    except (ValueError, OSError):
        return None

# List to hold all paper records
records = []

for paper in papers:
    record = {}
    content = paper.content
    
    # Extract paper-level details
    record['title'] = content.get('title', None)
    record['keywords'] = "; ".join(content.get('keywords', []))
    record['TL;DR'] = content.get('TL;DR', None)
    record['abstract'] = content.get('abstract', None)
    record['cdate'] = convert_timestamp(paper.cdate)
    record['tmdate'] = convert_timestamp(paper.tmdate)
    
    # Initialize reviewer fields
    for i in range(1, 5):
        prefix = f'reviewer{i}_'
        record[f'{prefix}rating_score'] = None
        record[f'{prefix}confidence_score'] = None
        record[f'{prefix}summary'] = None
        record[f'{prefix}strengths_and_weaknesses'] = None
        record[f'{prefix}questions'] = None
        record[f'{prefix}limitations'] = None
        record[f'{prefix}soundness'] = None
        record[f'{prefix}presentation'] = None
        record[f'{prefix}contribution'] = None
        record[f'{prefix}length'] = None
        record[f'{prefix}cdate'] = None
        record[f'{prefix}tmdate'] = None
    
    # Initialize decision field
    record['decision'] = None
    
    # Extract reviews and decision
    reviews = paper.details.get('directReplies', [])
    for reply in reviews:
        invitation = reply.get('invitation', '')
        content_reply = reply.get('content', {})
        
        if '/Official_Review' in invitation:
            # It's a review
            # Find the index based on existing filled reviewers
            reviewer_index = None
            for i in range(1, 5):
                prefix = f'reviewer{i}_'
                if record[f'{prefix}rating_score'] is None:
                    reviewer_index = i
                    break
            if reviewer_index is not None:
                prefix = f'reviewer{reviewer_index}_'
                record[f'{prefix}rating_score'] = content_reply.get('rating', None)
                record[f'{prefix}confidence_score'] = content_reply.get('confidence', None)
                record[f'{prefix}summary'] = content_reply.get('summary', None)
                record[f'{prefix}strengths_and_weaknesses'] = content_reply.get('strengths_and_weaknesses', None)
                record[f'{prefix}questions'] = content_reply.get('questions', None)
                record[f'{prefix}limitations'] = content_reply.get('limitations', None)
                record[f'{prefix}soundness'] = content_reply.get('soundness', None)
                record[f'{prefix}presentation'] = content_reply.get('presentation', None)
                record[f'{prefix}contribution'] = content_reply.get('contribution', None)
                
                # Calculate the length of the review
                review_text = ' '.join([
                    content_reply.get('summary', ''),
                    content_reply.get('strengths_and_weaknesses', ''),
                    content_reply.get('questions', ''),
                    content_reply.get('limitations', '')
                ])
                record[f'{prefix}length'] = len(review_text)
                
                # Extract review dates
                record[f'{prefix}cdate'] = convert_timestamp(reply.get('cdate', None))
                record[f'{prefix}tmdate'] = convert_timestamp(reply.get('tmdate', None))
        
        elif '/Decision' in invitation:
            # It's the decision
            record['decision'] = content_reply.get('decision', None)
    
    records.append(record)

# Create DataFrame
df = pd.DataFrame(records)

# Optional: Reorder columns for better readability
# Start with paper details
paper_columns = ['title', 'keywords', 'TL;DR', 'abstract', 'cdate', 'tmdate']
# Then reviewer details
reviewer_columns = []
for i in range(1, 5):
    prefix = f'reviewer{i}_'
    reviewer_columns.extend([
        f'{prefix}rating_score',
        f'{prefix}confidence_score',
        f'{prefix}summary',
        f'{prefix}strengths_and_weaknesses',
        f'{prefix}questions',
        f'{prefix}limitations',
        f'{prefix}soundness',
        f'{prefix}presentation',
        f'{prefix}contribution',
        f'{prefix}length',
        f'{prefix}cdate',
        f'{prefix}tmdate'
    ])
# Add decision as the last column
all_columns = paper_columns + reviewer_columns + ['decision']
df = df[all_columns]

# Export to CSV
df.to_csv("./data/" + conference_id.replace('/',"-") + '.csv', index=False)

print("CSV file 'papers_reviews_with_decision.csv' has been created successfully.")

CSV file 'papers_reviews_with_decision.csv' has been created successfully.


In [ ]:
import openreview
# client = openreview.api.OpenReviewClient(baseurl='https://api2.openreview.net')
client = openreview.Client(baseurl='https://api.openreview.net')

conference_id = 'ICLR.cc/2023/Conference'
papers = client.get_all_notes(
    invitation=f'{conference_id}/-/Blind_Submission',
    details='directReplies', 
)


Getting V1 Notes: 100%|█████████▉| 3792/3796 [00:06<00:00, 617.98it/s]


CSV file 'papers_reviews_with_decision.csv' has been created successfully.


In [202]:
papers[0].__dict__

{'id': 'zzqBoIFOQ1',
 'original': 'dC_9j4aLwcA',
 'number': 3283,
 'cdate': 1663850184200,
 'pdate': 1675279800000,
 'odate': 1664468100000,
 'mdate': None,
 'tcdate': 1663850184200,
 'tmdate': 1732526020341,
 'ddate': None,
 'content': {'title': 'Guiding Safe Exploration with Weakest Preconditions',
  'authorids': ['~Greg_Anderson1', '~Swarat_Chaudhuri1', '~Isil_Dillig1'],
  'authors': ['Greg Anderson', 'Swarat Chaudhuri', 'Isil Dillig'],
  'keywords': ['reinforcement learning', 'safe learning', 'safe exploration'],
  'TL;DR': 'We use an online, weakest-precondition-based approach to ensure safety during exploration without interfering with performance.',
  'abstract': 'In reinforcement learning for safety-critical settings, it is often desirable for the agent to obey safety constraints at all points in time, including during training. We present a novel neurosymbolic approach called SPICE to solve this safe exploration problem. SPICE uses an online shielding layer based on symbolic w

In [205]:
# for ICLR 2023, the decision is stored in the /Decision invitation
import pandas as pd
import json
from datetime import datetime


# Function to convert timestamp to readable date
def convert_timestamp(ts):
    if ts is None:
        return None
    try:
        return datetime.fromtimestamp(ts / 1000).strftime('%Y-%m-%d %H:%M:%S')
    except (ValueError, OSError):
        return None

# List to hold all paper records
records = []

for paper in papers:
    record = {}
    content = paper.content
    
    # Extract paper-level details
    record['title'] = content.get('title', None)
    record['keywords'] = "; ".join(content.get('keywords', []))
    record['TL;DR'] = content.get('TL;DR', None)
    record['abstract'] = content.get('abstract', None)
    record['cdate'] = convert_timestamp(paper.cdate)
    record['tmdate'] = convert_timestamp(paper.tmdate)
    
    # Initialize reviewer fields (up to 4 reviewers)
    for i in range(1, 5):
        prefix = f'reviewer{i}_'
        record[f'{prefix}recommendation'] = None
        record[f'{prefix}confidence'] = None
        record[f'{prefix}summary_of_the_paper'] = None
        record[f'{prefix}strength_and_weaknesses'] = None
        record[f'{prefix}clarity_quality_novelty_and_reproducibility'] = None
        record[f'{prefix}summary_of_the_review'] = None
        record[f'{prefix}correctness'] = None
        record[f'{prefix}technical_novelty_and_significance'] = None
        record[f'{prefix}empirical_novelty_and_significance'] = None
        record[f'{prefix}flag_for_ethics_review'] = None
        record[f'{prefix}length'] = None
        record[f'{prefix}cdate'] = None
        record[f'{prefix}tmdate'] = None

    # Initialize decision field
    record['decision'] = None

    # Extract reviews and decision
    reviews = paper.details.get('directReplies', [])
    for reply in reviews:
        invitation = reply.get('invitation', '')
        content_reply = reply.get('content', {})

        if '/Official_Review' in invitation:
            # It's a review
            # Find the first available reviewer slot
            reviewer_index = None
            for i in range(1, 5):
                prefix = f'reviewer{i}_'
                # Check if 'recommendation' is None to assign a new reviewer
                if record.get(f'{prefix}recommendation') is None:
                    reviewer_index = i
                    break
            if reviewer_index is not None:
                prefix = f'reviewer{reviewer_index}_'
                # Assign fields directly as per ICLR field names
                record[f'{prefix}recommendation'] = content_reply.get('recommendation', None)
                record[f'{prefix}confidence'] = content_reply.get('confidence', None)
                record[f'{prefix}summary_of_the_paper'] = content_reply.get('summary_of_the_paper', None)
                record[f'{prefix}strength_and_weaknesses'] = content_reply.get('strength_and_weaknesses', None)
                record[f'{prefix}clarity_quality_novelty_and_reproducibility'] = content_reply.get('clarity,_quality,_novelty_and_reproducibility', None)
                record[f'{prefix}summary_of_the_review'] = content_reply.get('summary_of_the_review', None)
                record[f'{prefix}correctness'] = content_reply.get('correctness', None)
                record[f'{prefix}technical_novelty_and_significance'] = content_reply.get('technical_novelty_and_significance', None)
                record[f'{prefix}empirical_novelty_and_significance'] = content_reply.get('empirical_novelty_and_significance', None)
                record[f'{prefix}flag_for_ethics_review'] = ", ".join(content_reply.get('flag_for_ethics_review', [])) if isinstance(content_reply.get('flag_for_ethics_review', []), list) else content_reply.get('flag_for_ethics_review', None)

                # Calculate the length of the review
                review_text = ' '.join([
                    content_reply.get('summary_of_the_paper', ''),
                    content_reply.get('strength_and_weaknesses', ''),
                    content_reply.get('clarity,_quality,_novelty_and_reproducibility', ''),
                    content_reply.get('summary_of_the_review', ''),
                    content_reply.get('correctness', ''),
                    content_reply.get('technical_novelty_and_significance', ''),
                    content_reply.get('empirical_novelty_and_significance', ''),
                ])
                record[f'{prefix}length'] = len(review_text)

                # Extract review dates
                record[f'{prefix}cdate'] = convert_timestamp(reply.get('cdate', None))
                record[f'{prefix}tmdate'] = convert_timestamp(reply.get('tmdate', None))
        
        elif '/Decision' in invitation:
            # It's the decision
            record['decision'] = content_reply.get('decision', None)
    
    records.append(record)

# Create DataFrame
df = pd.DataFrame(records)

# Optional: Reorder columns for better readability
# Start with paper details
paper_columns = ['title', 'keywords', 'TL;DR', 'abstract', 'cdate', 'tmdate']
# Then reviewer details
reviewer_columns = []
for i in range(1, 5):
    prefix = f'reviewer{i}_'
    reviewer_columns.extend([
        f'{prefix}recommendation',
        f'{prefix}confidence',
        f'{prefix}summary_of_the_paper',
        f'{prefix}strength_and_weaknesses',
        f'{prefix}clarity_quality_novelty_and_reproducibility',
        f'{prefix}summary_of_the_review',
        f'{prefix}correctness',
        f'{prefix}technical_novelty_and_significance',
        f'{prefix}empirical_novelty_and_significance',
        f'{prefix}flag_for_ethics_review',
        f'{prefix}length',
        f'{prefix}cdate',
        f'{prefix}tmdate'
    ])
# Add decision as the last column
all_columns = paper_columns + reviewer_columns + ['decision']
df = df[all_columns]

# Export to CSV
df.to_csv("./data/" + conference_id.replace('/',"-") + '.csv', index=False)

print("CSV file 'papers_reviews_with_decision.csv' has been created successfully.")

CSV file 'papers_reviews_with_decision.csv' has been created successfully.
